#Intro
First of all thanks to Kjetil Åmdal-Sævik for providing excellent code for data preparation.
Being a novice python programmer, my code may not be that much efficient but it may serve as a starting point for using TensorFlow.

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import random
import math
import warnings
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [ ]:
# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = '../input/stage1_train/'
TEST_PATH = '../input/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [ ]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [ ]:
# Get and resize train images and masks
images = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
labels = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    images[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    labels[n] = mask

X_train = images
Y_train = labels
Y_train = Y_train.astype(np.float32)

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

In [ ]:
def shuffle():
    global images, labels
    p = np.random.permutation(len(X_train))
    images = X_train[p]
    labels = Y_train[p]

In [ ]:
def next_batch(batch_s, iters):
    if(iters == 0):
        shuffle()
    count = batch_s * iters
    return images[count:(count + batch_s)], labels[count:(count + batch_s)]

In [ ]:
def deconv2d(input_tensor, filter_size, output_size, out_channels, in_channels, name, strides = [1, 2, 2, 1]):
    dyn_input_shape = tf.shape(input_tensor)
    batch_size = dyn_input_shape[0]
    out_shape = tf.stack([batch_size, output_size, output_size, out_channels])
    filter_shape = [filter_size, filter_size, out_channels, in_channels]
    w = tf.get_variable(name=name, shape=filter_shape)
    h1 = tf.nn.conv2d_transpose(input_tensor, w, out_shape, strides, padding='VALID')
    return h1

In [ ]:
X = tf.placeholder(tf.float32, [None, 128, 128, 3])
Y_ = tf.placeholder(tf.float32, [None, 128, 128, 1])
lr = tf.placeholder(tf.float32)

In [ ]:
Y0 = tf.layers.conv2d(X, filters=16, kernel_size=1, strides=1, padding="VALID", activation=tf.nn.relu)
Y1 = tf.layers.conv2d(Y0, filters=32, kernel_size=2, strides=2, padding="VALID", activation=tf.nn.relu)
Y2 = tf.layers.conv2d(Y1, filters=64, kernel_size=2, strides=2, padding="VALID", activation=tf.nn.relu)

Y3_ = deconv2d(Y2, 1, 32, 32, 64, "Y3_deconv", strides=[1, 1, 1, 1])
Y3_ = tf.nn.relu(Y3_)

Y2_ = deconv2d(Y3_, 2, 64, 16, 32, "Y2_deconv")
Y2_ = tf.nn.relu(Y2_)

Y1_ = deconv2d(Y2_, 2, 128, 8, 16, "Y1_deconv")
Y1_ = tf.nn.relu(Y1_)

Y0_ = deconv2d(Y1_, 1, 128, 1, 8, "Y0_deconv", strides=[1, 1, 1, 1])

logits = Y0_
loss = tf.reduce_mean(tf.square(Y_ - logits))

In [ ]:
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)
# init
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

iter_count = 0

for i in range(15000):
    # training on batches of 5 images with 5 mask images
    if(iter_count > 114):
        iter_count = 0    

    batch_X, batch_Y = next_batch(5, iter_count)

    iter_count += 1

    feed_dict = {X: batch_X, Y_: batch_Y, lr: 0.0005}
    loss_value = sess.run([loss], feed_dict=feed_dict)

    if(i % 500 == 0):
        print("training loss:", str(loss_value))

print("Done!")

Test on the data that is not seen by the network during training:

In [ ]:
ix = 20 #random.randint(0, 64) #len(X_test) - 1 = 64
test_image = X_test[ix].astype(float)
imshow(test_image)
plt.show()

In [ ]:
#print(ix)
test_image = np.reshape(test_image, [-1, 128 , 128, 3])
test_data = {X:test_image}

test_mask = sess.run([logits],feed_dict=test_data)
test_mask = np.reshape(test_mask, [128 , 128, 1])
imshow(test_mask.squeeze(), cmap="gray_r")
plt.show()